In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import torch

In [2]:
from utils.transformer_character_embeddings import embeddings_from_text

/home/cat/EPFL/Fall 2023/ADA/project_repo/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
plots =  pd.read_csv(
    'data/MovieSummaries/plot_summaries.txt', 
    sep='\t', 
    names=['wiki_id', 'plot']
)

plots['plot'] = plots['plot'].apply(lambda x: ' '.join(x.split()))

plots.head(5)

,wiki_id,plot
0,23890098,"Shlykov, a hard-working taxi driver and Lyosha..."
1,31186339,The nation of Panem consists of a wealthy Capi...
2,20663735,Poovalli Induchoodan is sentenced for six year...
3,2231378,"The Lemon Drop Kid , a New York City swindler,..."
4,595909,Seventh-day Adventist Church pastor Michael Ch...


In [4]:
input_text = plots.iloc[1]['plot']

In [7]:
character_list = []

for index, row in tqdm([row for row in plots.iterrows()][:10]):
    plot = row['plot'] 
    character_emb = embeddings_from_text(plot)

    for name in character_emb.keys():
        character_list.append(
            {
                'wiki_id': row['wiki_id'],
                'character': name,
                'emb': character_emb[name].tolist(),
            }
        )

character_df = pd.DataFrame(character_list)
character_df.head()

100%|██████████| 10/10 [00:44<00:00,  4.46s/it]


,wiki_id,character,emb
0,31186339,Primrose Everdeen,"[-0.30263614654541016, -0.6969847679138184, 0...."
1,31186339,Katniss,"[-0.19846008718013763, -0.7502771019935608, 0...."
2,31186339,Peeta Mellark,"[-0.10818104445934296, -0.3706214129924774, -0..."
3,31186339,Haymitch Abernathy,"[-0.1719408631324768, -0.26161321997642517, -0..."
4,31186339,Caesar Flickerman,"[0.8750091791152954, -0.30963605642318726, -0...."


In [71]:
from sklearn.cluster import DBSCAN
X = np.array(character_df['emb'].values.tolist())
clustering = DBSCAN(eps=0.4, min_samples=1, metric='cosine').fit(X)
clustering.labels_

array([ 0,  0,  0,  1,  0,  2,  0,  3,  4,  5,  5,  5,  6,  5,  5,  5,  5,
        5,  5,  5,  5,  5,  5,  5,  7,  8,  8,  9,  9, 10, 11,  8,  0,  0,
       12,  8,  8, 13,  8,  8,  8, 14,  0, 15,  8, 16,  8,  8,  8,  0,  8,
        8,  8, 17,  8, 18, 19, 20,  8, 21])

In [72]:
character_df['character'].iloc[clustering.labels_==0]

0     Primrose Everdeen
1               Katniss
2         Peeta Mellark
4     Caesar Flickerman
6                  Cato
32                Lindy
33               Azaria
42               Dahlia
49             Lombardo
Name: character, dtype: object

In [73]:
character_df['character'].iloc[clustering.labels_==1]

3    Haymitch Abernathy
Name: character, dtype: object

In [75]:
character_df['character'].iloc[clustering.labels_==2]

5    Rue
Name: character, dtype: object

In [76]:
character_df['character'].iloc[clustering.labels_==3]

7    Snow
Name: character, dtype: object

In [77]:
character_df['character'].iloc[clustering.labels_==8]

25            Moose Moran
26          Brainy Baxter
31    Michael Chamberlain
35            Alex Thomas
36          Kate Crawford
38           Thomas jumps
39           Jack Baldwin
40        Vaughan Stevens
44                Cecilia
46                 Murray
47         Natasha Rimsky
48                 Hannah
50                Dominic
51                 Mickey
52           Ann Mitchell
54        John Willoughby
58           Buzz Buzzard
Name: character, dtype: object